# 项目：整理Netflix电影演员评分数据

## 分析目标

此数据分析的目的是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

本实战项目的目的在于练习整理数据，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了截止至2022年7月美国地区可观看的所有Netflix电视剧及电影数据。数据集包含两个数据表：`titles.csv`和`credits.csv`。

`titles.csv`包含电影及电视剧相关信息，包括影视作品ID、标题、类型、描述、流派、IMDB（一个国外的在线评分网站）评分，等等。`credits.csv`包含超过7万名出现在Netflix影视作品的导演及演员信息，包括名字、影视作品ID、人物名、演职员类型（导演/演员）等。

`titles.csv`每列的含义如下：
- id：影视作品ID。
- title：影视作品标题。
- show_type：作品类型，电视节目或电影。
- description：简短描述。
- release_year：发布年份。
- age_certification：适龄认证。
- runtime：每集电视剧或电影的长度。
- genres：流派类型列表。
- production_countries：出品国家列表。
- seasons：如果是电视剧，则是季数。
- imdb_id：IMDB的ID。
- imdb_score：IMDB的评分。
- imdb_votes：IMDB的投票数。
- tmdb_popularity：TMDB的流行度。
- tmdb_score：TMDB的评分。

`credits.csv`每列的含义如下：
- person_ID：演职员ID。
- id：参与的影视作品ID。
- name：姓名。
- character_name：角色姓名。
- role：演职员类型，演员或导演。

In [1]:
import pandas as pd
import numpy as np

In [2]:
raw_titles = pd.read_csv("titles.csv")
raw_credits = pd.read_csv("credits.csv")

# 数据整齐度

先查看raw_titles的数据整齐度

In [3]:
raw_titles.head(5)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


可以看到其中的genres 和 production_countries两个变量中包含多个值，应当进行拆分

拆分genres

先提取任意一个genres变量的值进行观察

In [4]:
raw_titles["genres"][1]

"['drama', 'crime']"

genres实际类型并非字符串列表，而是字符串,可以使用Python内置的eval函数，它可以把字符串转换成表达式

In [5]:
raw_titles["genres"] = raw_titles["genres"].apply(lambda s :eval(s))
raw_titles["genres"][1]

['drama', 'crime']

然后进行拆分

In [6]:
raw_titles = raw_titles.explode("genres")
raw_titles.head(5)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300


可以看到genres已经拆分完成

## 拆分production_countries

先提取任意一个production_countries变量的值进行观察

In [7]:
raw_titles["production_countries"][1]

1    ['US']
1    ['US']
Name: production_countries, dtype: object

production_countries实际类型并非字符串列表，而是字符串,可以使用Python内置的eval函数，它可以把字符串转换成表达式

In [8]:
raw_titles["production_countries"] = raw_titles["production_countries"].apply(lambda s :eval(s))

转换完成后，开始拆分production_countries

In [9]:
raw_titles = raw_titles.explode("production_countries")
raw_titles.head(5)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,US,NaN,tt0068473,7.7,107673.0,10.010,7.300


可以看到，已经对production_countries拆分完成

查看raw_read_csv的数据整齐度

In [10]:
raw_credits.head(5)

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


不存在有同一变量中存在多个值的情况

# 数据干净度

评估raw_titles的干净度

In [11]:
raw_titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17818 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    17818 non-null  object 
 1   title                 17817 non-null  object 
 2   type                  17818 non-null  object 
 3   description           17790 non-null  object 
 4   release_year          17818 non-null  int64  
 5   age_certification     10889 non-null  object 
 6   runtime               17818 non-null  int64  
 7   genres                17755 non-null  object 
 8   production_countries  17439 non-null  object 
 9   seasons               6224 non-null   float64
 10  imdb_id               17116 non-null  object 
 11  imdb_score            16976 non-null  float64
 12  imdb_votes            16945 non-null  float64
 13  tmdb_popularity       17663 non-null  float64
 14  tmdb_score            17241 non-null  float64
dtypes: float64(5), int64(2), 

总共有17818行数据，其中 title 、description、age_certification 、genres、production_countries、seasons 、 imdb_id 、imdb_score 、imdb_votes 、tmdb_popularity 、tmdb_score都存在缺失值。其中release_year 的类型应该为日期类型，要进行转换

release_year 类型转换

In [12]:
raw_titles["release_year"] = pd.to_datetime(raw_titles["release_year"])

由于影视作品的标题、描述、适龄认证、发行国家、电视剧季数、IMDB的ID、TMDB的流行度、TMDB的评分，并不影响我们挖掘各个流派中的高IMDB评分作品演员，所以可以保留title、description、age_certification、production_countries、seasons、imdb_id、tmdb_popularity、tmdb_score、imdb_votes、tmdb_popularity、tmdb_score变量值存在空缺的观察值。

但imdb_score和genres，即IMDB评分和流派，和我们后续要做的分析息息相关。

先提取imdb_score的缺失值

In [13]:
raw_titles[raw_titles["imdb_score"].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1970-01-01 00:00:00.000001945,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
75,tm132164,Bill Hicks: Sane Man,MOVIE,Sane Man was filmed before Bill recorded ‘Dang...,1970-01-01 00:00:00.000001989,R,80,comedy,US,NaN,NaN,NaN,NaN,3.377,7.5
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1970-01-01 00:00:00.000001991,TV-G,18,documentation,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1970-01-01 00:00:00.000001991,TV-G,18,family,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1970-01-01 00:00:00.000001991,TV-G,18,reality,JP,12.0,NaN,NaN,NaN,7.730,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5810,tm1225897,Social Man,MOVIE,Two competitive social media Influencers go he...,1970-01-01 00:00:00.000002021,NaN,96,drama,NaN,NaN,tt20198164,NaN,NaN,NaN,NaN
5833,ts307884,HQ Barbers,SHOW,When a family run barber shop in the heart of ...,1970-01-01 00:00:00.000002021,TV-14,24,comedy,NG,1.0,NaN,NaN,NaN,0.840,NaN
5840,tm1216735,Sun of the Soil,MOVIE,"In 14th-century Mali, an ambitious young royal...",1970-01-01 00:00:00.000002022,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,1.179,7.0
5844,tm1074617,Bling Empire - The Afterparty,MOVIE,"The stars of ""Bling Empire"" discuss the show's...",1970-01-01 00:00:00.000002021,NaN,35,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN


由于imdb_score是我们分析数据的核心元素，因此把缺失的部分直接删除掉

In [14]:
raw_titles = raw_titles.dropna(subset=["imdb_score"])
raw_titles["imdb_score"].isnull().sum()

0

提取genres的缺失值

In [15]:
raw_titles[raw_titles["genres"].isnull()]

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1813,ts77824,My Next Guest Needs No Introduction With David...,SHOW,TV legend David Letterman teams up with fascin...,1970-01-01 00:00:00.000002018,TV-MA,50,NaN,US,4.0,tt7829834,7.8,5581.0,8.217,7.6
1939,ts215037,Minecraft: Story Mode,SHOW,"MInecraft: Story Mode is an interactive, anima...",1970-01-01 00:00:00.000002018,TV-PG,52,NaN,US,1.0,tt10498322,5.6,347.0,NaN,NaN
2386,ts74805,A Little Help with Carol Burnett,SHOW,In this unscripted series starring comedy lege...,1970-01-01 00:00:00.000002018,TV-G,24,NaN,US,1.0,tt7204366,6.3,237.0,1.621,6.2
2658,ts265844,#ABtalks,SHOW,#ABtalks is a YouTube interview show hosted by...,1970-01-01 00:00:00.000002018,TV-PG,68,NaN,NaN,1.0,tt12635254,9.6,7.0,NaN,NaN
4274,tm1172010,The Lockdown Plan,MOVIE,NaN,1970-01-01 00:00:00.000002020,NaN,49,NaN,NaN,NaN,tt13079112,6.5,NaN,NaN,NaN
4648,tm1113921,In Vitro,MOVIE,'In Vitro' is an otherworldly rumination on me...,1970-01-01 00:00:00.000002019,NaN,27,NaN,NaN,NaN,tt10545994,7.7,NaN,NaN,NaN


由于 genres 是我们分析数据的核心元素，因此把缺失的部分直接删除掉

In [16]:
raw_titles = raw_titles.dropna(subset=["genres"])
raw_titles["genres"].isnull().sum()

0

评估 raw_credits 的干净度


In [17]:
raw_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


总共77801行数据，其中character  存在缺失值。person_id  为演员ID，应该为文本类型，需要转换类型

person_id类型转换

In [18]:
raw_credits["person_id"] = raw_credits["person_id"].astype(str)
raw_credits["person_id"]

0           3748
1          14658
2           7064
3           3739
4          48933
          ...   
77796     736339
77797     399499
77798     373198
77799     378132
77800    1950416
Name: person_id, Length: 77801, dtype: object

查看character 存在的缺失值

In [19]:
raw_credits[raw_credits["character"].isnull()]

,person_id,id,name,character,role
36,3308,tm84618,Martin Scorsese,NaN,DIRECTOR
59,17727,tm154986,John Boorman,NaN,DIRECTOR
106,11475,tm127384,Terry Jones,NaN,DIRECTOR
107,11473,tm127384,Terry Gilliam,NaN,DIRECTOR
162,1063,tm120801,Robert Aldrich,NaN,DIRECTOR
...,...,...,...,...,...
77776,2363022,tm1097142,Mohamed El-Arkan,NaN,ACTOR
77777,1827884,tm1097142,Mohamed Bakir,NaN,DIRECTOR
77783,678884,tm1014599,Segun Arinze,NaN,ACTOR
77789,1962840,tm1014599,Seyi Babatope,NaN,DIRECTOR


character 为角色名，不影响我们做数据分析，因此对于此缺失值可以保留

# 评估重复数据

raw_titles 中不应该存在每个变量都有重复的值，查看是否存在重复

In [20]:
raw_titles.duplicated().sum()

0

raw_titles中不存在重复的行

raw_credits也是如此

In [21]:
raw_credits.duplicated().sum()

0

raw_credits 中不存在重复的行

# 评估不一致数据

针对raw_titles 中可能的不一致数据为 genres 中不同值指代了同一种流派的情况,以及 production_countries 中不同值代表了不同国家的情况

In [22]:
raw_titles["genres"].value_counts()

genres
drama            3357
comedy           2419
thriller         1446
action           1339
romance          1080
crime            1066
documentation     981
family            769
animation         732
fantasy           727
european          679
scifi             647
horror            438
history           336
music             266
reality           226
war               221
sport             188
western            53
Name: count, dtype: int64

从输出上看，不存在不同值指代同一流派的情况

production_countries 中不同值代表了不同国家的情况

In [23]:
raw_titles["production_countries"].value_counts()

production_countries
US    5648
IN    1610
GB    1068
JP    1046
FR     720
      ... 
GT       1
CU       1
LK       1
NP       1
FO       1
Name: count, Length: 108, dtype: int64

In [24]:
with pd.option_context('display.max_rows', None):
    print(raw_titles['production_countries'].value_counts())

production_countries
US         5648
IN         1610
GB         1068
JP         1046
FR          720
KR          637
ES          637
CA          608
DE          383
CN          295
MX          264
IT          224
BR          221
AU          217
TR          195
PH          192
AR          150
ID          149
BE          148
TW          133
NG          131
PL          126
ZA          103
NL          102
HK          102
CO           94
EG           93
DK           89
TH           87
SE           81
LB           70
NO           68
AE           52
IE           49
SG           47
XX           43
IL           42
RU           41
CL           35
CH           33
PS           32
BG           31
MY           30
SA           28
AT           28
IS           28
LU           27
NZ           27
PE           26
RO           25
QA           24
CZ           22
JO           19
FI           18
HU           18
UY           15
MA           15
PT           14
KH           10
KW           10
PR            9
PK 

从列表中可以看出每个国家都有两个大写字母表示，但其中Lebanon和LB重复，将Lebanon替换为LB

In [25]:
raw_titles["production_countries"] = raw_titles["production_countries"].replace({"Lebanon":"LB"})
with pd.option_context('display.max_rows', None):
    print(raw_titles['production_countries'].value_counts())

production_countries
US    5648
IN    1610
GB    1068
JP    1046
FR     720
KR     637
ES     637
CA     608
DE     383
CN     295
MX     264
IT     224
BR     221
AU     217
TR     195
PH     192
AR     150
ID     149
BE     148
TW     133
NG     131
PL     126
ZA     103
HK     102
NL     102
CO      94
EG      93
DK      89
TH      87
SE      81
LB      71
NO      68
AE      52
IE      49
SG      47
XX      43
IL      42
RU      41
CL      35
CH      33
PS      32
BG      31
MY      30
AT      28
SA      28
IS      28
LU      27
NZ      27
PE      26
RO      25
QA      24
CZ      22
JO      19
FI      18
HU      18
UY      15
MA      15
PT      14
KW      10
KH      10
PK       9
PR       9
UA       8
MT       8
VN       8
LT       7
IR       7
CD       7
SU       7
TN       7
SN       6
AL       6
KE       6
GH       6
IQ       5
MU       5
CY       5
KN       4
GR       4
IO       4
SY       4
TZ       4
MC       4
GL       3
CM       3
AO       3
BS       3
HR       3
BD       3


不存在不一致的值了

针对 raw_credits 中可能存在不一致的变量 role ：演职员类型，演员或导演。

In [26]:
raw_credits["role"].value_counts()

role
ACTOR       73251
DIRECTOR     4550
Name: count, dtype: int64

从结果上看，role 只存在两个类型，因此不存在不一致的的值指代同中演职员类型

# 处理无效或错误数据

针对 raw_titles 

In [27]:
raw_titles.describe()

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,16970,16970.000000,5954.000000,16970.000000,1.694100e+04,16842.000000,16515.000000
mean,1970-01-01 00:00:00.000002015,80.912552,2.455492,6.514207,3.281655e+04,29.396307,6.846933
min,1970-01-01 00:00:00.000001954,0.000000,1.000000,1.500000,5.000000e+00,0.600000,1.000000
25%,1970-01-01 00:00:00.000002015,45.000000,1.000000,5.800000,7.800000e+02,4.070000,6.200000
50%,1970-01-01 00:00:00.000002018,90.000000,2.000000,6.600000,3.508000e+03,10.195000,6.900000
75%,1970-01-01 00:00:00.000002020,107.000000,3.000000,7.300000,1.697800e+04,23.639000,7.500000
max,1970-01-01 00:00:00.000002022,225.000000,42.000000,9.500000,2.294231e+06,2274.044000,10.000000
std,NaN,39.596172,2.869428,1.131095,1.141492e+05,93.178235,1.078831


raw_titles 不存在无意义的值

针对raw_credits

raw_credits 由于不包含表示数值含义的变量，因此无需用describe检查。

# 整理数据

In [28]:
raw_titles

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1970-01-01 00:00:00.000001972,R,109,drama,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1970-01-01 00:00:00.000001972,R,109,action,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1970-01-01 00:00:00.000001972,R,109,thriller,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5846,tm898842,C/O Kaadhal,MOVIE,A heart warming film that explores the concept...,1970-01-01 00:00:00.000002021,NaN,134,drama,NaN,NaN,tt11803618,7.7,348.0,NaN,NaN
5847,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,1970-01-01 00:00:00.000002021,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",1970-01-01 00:00:00.000002021,NaN,7,family,NaN,1.0,tt13711094,7.8,18.0,2.289,10.000
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",1970-01-01 00:00:00.000002021,NaN,7,animation,NaN,1.0,tt13711094,7.8,18.0,2.289,10.000


In [29]:
raw_credits

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR
...,...,...,...,...,...
77796,736339,tm1059008,Adelaida Buscato,María Paz,ACTOR
77797,399499,tm1059008,Luz Stella Luengas,Karen Bayona,ACTOR
77798,373198,tm1059008,Inés Prieto,Fanny,ACTOR
77799,378132,tm1059008,Isabel Gaona,Cacica,ACTOR


此数据分析的目的是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。为了能同时获取流派以及演员的数据，我们需要将两个表通过 id 键进行拼接，因为两个表的 id 都同时表示影视作品的ID。用 tiltes_with_credits 来命名合并后的 DataFrame 。

In [30]:
tiltes_with_credits = pd.merge(raw_titles,raw_credits, on = "id", how = "inner")
tiltes_with_credits

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,3748,Robert De Niro,Travis Bickle,ACTOR
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,14658,Jodie Foster,Iris Steensma,ACTOR
2,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,7064,Albert Brooks,Tom,ACTOR
3,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,3739,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,48933,Cybill Shepherd,Betsy,ACTOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276104,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,1970-01-01 00:00:00.000002021,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300,736339,Adelaida Buscato,María Paz,ACTOR
276105,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,1970-01-01 00:00:00.000002021,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300,399499,Luz Stella Luengas,Karen Bayona,ACTOR
276106,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,1970-01-01 00:00:00.000002021,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300,373198,Inés Prieto,Fanny,ACTOR
276107,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,1970-01-01 00:00:00.000002021,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300,378132,Isabel Gaona,Cacica,ACTOR


因为我们只是挖掘出各个流派中的高评分作品演员，和导演没有关系，因此应该筛选出 role 中的 ACTOR 来分析数据

In [31]:
tiltes_with_credits = tiltes_with_credits.query('role == "ACTOR"')
tiltes_with_credits

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score,person_id,name,character,role
0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,3748,Robert De Niro,Travis Bickle,ACTOR
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,14658,Jodie Foster,Iris Steensma,ACTOR
2,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,7064,Albert Brooks,Tom,ACTOR
3,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,3739,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1970-01-01 00:00:00.000001976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179,48933,Cybill Shepherd,Betsy,ACTOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276103,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,1970-01-01 00:00:00.000002021,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300,368473,Aída Morales,Maritza,ACTOR
276104,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,1970-01-01 00:00:00.000002021,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300,736339,Adelaida Buscato,María Paz,ACTOR
276105,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,1970-01-01 00:00:00.000002021,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300,399499,Luz Stella Luengas,Karen Bayona,ACTOR
276106,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,1970-01-01 00:00:00.000002021,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300,373198,Inés Prieto,Fanny,ACTOR


由于是通过 genres 流派和 person_id 演员的 imdb_score 来挖掘出各个流派中的高评分作品演员，因此我们需要对流派和演员进行分组

In [32]:
genrs_and_person_id = tiltes_with_credits.groupby(["genres","person_id"])

然后我们就可以根据 imdb_score 来计算平均值

In [33]:
imdb_score_with_genrs_and_person_id = genrs_and_person_id["imdb_score"].mean()
imdb_score_with_genrs_and_person_id

genres   person_id
action   1000         6.866667
         100007       7.000000
         100013       6.400000
         100019       6.500000
         100020       6.500000
                        ...   
western  993735       6.500000
         998673       7.300000
         998674       7.300000
         998675       7.300000
         99940        4.000000
Name: imdb_score, Length: 168881, dtype: float64

为了更准确的分析，可以用 reset_index() 消除层次化索引

In [34]:
imdb_score_with_genrs_and_person_id = imdb_score_with_genrs_and_person_id.reset_index()

找出各个流派中最高评分是多少

In [35]:
genres_imdb_score_max =imdb_score_with_genrs_and_person_id.groupby("genres")["imdb_score"].max()
genres_imdb_score_max 

genres
action           9.3
animation        9.3
comedy           9.2
crime            9.5
documentation    9.1
drama            9.5
european         8.9
family           9.3
fantasy          9.3
history          9.1
horror           9.0
music            8.8
reality          8.9
romance          9.2
scifi            9.3
sport            9.1
thriller         9.5
war              8.8
western          8.9
Name: imdb_score, dtype: float64

可以根据上表与之前的imdb_score_with_genrs_and_person_id合并，可以得到最高分的演员ID

In [36]:
genres_imdb_score_max_person_id = pd.merge(genres_imdb_score_max, imdb_score_with_genrs_and_person_id, on = ["genres","imdb_score"])
genres_imdb_score_max_person_id

,genres,imdb_score,person_id
0,action,9.3,12790
1,action,9.3,1303
2,action,9.3,21033
3,action,9.3,336830
4,action,9.3,86591
...,...,...,...
131,war,8.8,826547
132,western,8.9,22311
133,western,8.9,28166
134,western,8.9,28180


由于我们只需要得到挖掘出各个流派中的高评分作品演员，所以我们只要 raw_credits 中演员id和演员name就可以了，重复的删除

In [37]:
person_id_and_name = raw_credits[["person_id", "name"]].drop_duplicates()
person_id_and_name

,person_id,name
0,3748,Robert De Niro
1,14658,Jodie Foster
2,7064,Albert Brooks
3,3739,Harvey Keitel
4,48933,Cybill Shepherd
...,...,...
77796,736339,Adelaida Buscato
77797,399499,Luz Stella Luengas
77798,373198,Inés Prieto
77799,378132,Isabel Gaona


然后再将 genres_imdb_score_max_person_id 和 person_id_and_name 合并就可以得到评分对应的演员名字name

In [40]:
genres_max_score_with_name = pd.merge(genres_imdb_score_max_person_id,person_id_and_name, on= ["person_id"])
genres_max_score_with_name

,genres,imdb_score,person_id,name
0,action,9.3,12790,Olivia Hack
1,scifi,9.3,12790,Olivia Hack
2,action,9.3,1303,Jessie Flower
3,animation,9.3,1303,Jessie Flower
4,family,9.3,1303,Jessie Flower
...,...,...,...,...
131,war,8.8,826547,Yuto Uemura
132,western,8.9,22311,Koichi Yamadera
133,western,8.9,28166,Megumi Hayashibara
134,western,8.9,28180,Unsho Ishizuka


从输出来看， genres 还是杂乱排序的，把它重新排序

In [42]:
genres_max_score_with_name.sort_values("genres")

,genres,imdb_score,person_id,name
0,action,9.3,12790,Olivia Hack
12,action,9.3,336830,André Sogliuzzo
7,action,9.3,21033,Zach Tyler
17,action,9.3,86591,Cricket Leigh
2,action,9.3,1303,Jessie Flower
...,...,...,...,...
131,war,8.8,826547,Yuto Uemura
133,western,8.9,28166,Megumi Hayashibara
134,western,8.9,28180,Unsho Ishizuka
132,western,8.9,22311,Koichi Yamadera


在把索引重新排序,并且删掉之前的 index

In [48]:
 genres_max_score_with_name = genres_max_score_with_name.sort_values("genres").reset_index().drop("index", axis =1 )
genres_max_score_with_name

,genres,imdb_score,person_id,name
0,action,9.3,12790,Olivia Hack
1,action,9.3,336830,André Sogliuzzo
2,action,9.3,21033,Zach Tyler
3,action,9.3,86591,Cricket Leigh
4,action,9.3,1303,Jessie Flower
...,...,...,...,...
131,war,8.8,826547,Yuto Uemura
132,western,8.9,22311,Koichi Yamadera
133,western,8.9,28166,Megumi Hayashibara
134,western,8.9,28180,Unsho Ishizuka


## 得到了各个流派中评分最好的演员